# Реализация модели защищенного канала передачи данных по спецификации TLS 1.3

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

----

In [3]:
# Криптография из различных ГОСТ'ов
from pygost import mgm
from pygost import gost3412
from pygost import gost3410
from pygost import gost34112012256

# Криптография из западных стандартов
from Crypto.Cipher import AES
from Crypto.Hash import SHA256

# HKDF
from Crypto.Protocol.KDF import HKDF

# КГПСЧ
from Crypto.Random import get_random_bytes, random
from Crypto.Util.number import getStrongPrime

# Различные утилиты
from collections import namedtuple
from Crypto.Util import number
from functools import partial
from binascii import hexlify
from enum import Enum
import uuid
import gmpy2
import pickle

----

## Реализация оберток над криптографическими алгоритмами

In [4]:
def multiply_by_scalar(curve, scalar, point):
    """
    Умножение точки эллиптической кривой на скаляр 
    """
    
    return curve.exp(scalar, *point)


def add_points(curve, lhs, rhs):
    """
    Сложение двух точек эллиптической кривой
    """
    
    return curve._add(*lhs, *rhs)

---

In [5]:
#
# Сертификат это простая структурка, поэтому обойдусь именованным кортежем
#

Certificate = namedtuple('Certificate', ['id', 'public_key', 'signature'])

----

In [6]:
class KuznyechikMGM(object):
    """
    Класс-обертка над Кузнечиком в режиме MGM
    """
    
    KEY_SIZE = 32
    
    def __init__(self, key: bytes):
        """
        Инициализация - сохраняю ключ
        """

        self._key = key
    
    
    def encrypt(self, plaintext: bytes):
        """
        Зашифрование шифром Кузнечик в режиме MGM
        """
        
        encrypter = partial(KuznyechikMGM._encrypt_block, self._key)
        cipher = mgm.MGM(encrypter, KuznyechikMGM._block_size())
        
        nonce = KuznyechikMGM._generate_nonce()
        ct_with_tag = cipher.seal(nonce, plaintext, b'')
        
        return nonce, ct_with_tag[:-cipher.tag_size], ct_with_tag[-cipher.tag_size:]
    
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes):
        """
        Расшифрование шифром Кузнечик в режиме MGM
        """
        
        encrypter = partial(KuznyechikMGM._encrypt_block, self._key)
        cipher = mgm.MGM(encrypter, KuznyechikMGM._block_size())
            
        ct_with_tag = ciphertext + tag
            
        return cipher.open(nonce, ct_with_tag, b'')
        
        
    @staticmethod
    def _block_size():
        return gost3412.GOST3412Kuznechik.blocksize
        
        
    @staticmethod
    def _encrypt_block(key: bytes, block: bytes):
        cipher = gost3412.GOST3412Kuznechik(key)
        return cipher.encrypt(block)
    
    
    @staticmethod
    def _generate_nonce():
        nonce = get_random_bytes(KuznyechikMGM._block_size())
        while bytearray(nonce)[0] & 0x80 > 0:
            nonce = get_random_bytes(KuznyechikMGM._block_size())
            
        return nonce

In [7]:
class AESGCM(object):
    """
    Класс-обертка над AES в режиме GCM
    """
    
    KEY_SIZE = 16
    
    def __init__(self, key: bytes):
        """
        Инициализация - сохраняю ключ
        """

        self._key = key
        
        
    def encrypt(self, plaintext: bytes):
        """
        Зашифрование шифром AES в режиме GCM
        """
        
        cipher = AES.new(self._key, AES.MODE_GCM)
        return cipher.nonce, *cipher.encrypt_and_digest(plaintext)
    
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes):
        """
        Зашифрование шифром AES в режиме GCM
        """
        
        cipher = AES.new(self._key, AES.MODE_GCM, nonce=nonce)
        return cipher.decrypt_and_verify(ciphertext, tag)

----

In [8]:
class Streebog(gost34112012256.GOST34112012256):
    """
    pycryptodome-совместимая обертка над хэш-функцией Стрибог
    """
    
    digest_size = 32
    
    def __init__(self, data=None):
        """
        Инициализация и обновление состояния (если надо)
        """
        
        super(Streebog, self).__init__()
        if data is not None:
            self.update(data)
    
    @staticmethod
    def new(data=None):
        """
        Создание нового объекта
        """
        
        return Streebog(data)

In [9]:
def hash_digest(hasher_type, *args):
    """
    Вычисление хэша при помощи объекта-обертки
    """
    
    hasher = hasher_type.new()
    
    for arg in args:
        hasher.update(arg)
        
    return hasher.digest()

----

In [10]:
class DiffieHellmanProcessor(object):
    """
    Обработчик обмена ключами по протоколу Диффи-Хеллмана
    """
    
    def __init__(self, group=None, generator=None):
        """
        Инициализация - выбор группы, образующего элемента
        Возможно конструирование по готовой группе и образующему
        """

        self._group = Integers(getStrongPrime(512)) if group is None else group
        self._generator = random.randint(int(2), int(self._group.order() - 2)) if generator is None \
            else generator
        
        self._modulus = self._group.order()
        
        trace('[DiffieHellmanProcessor]', f'Using group of order {self._modulus}')
        
    
    def generate_private_key(self):
        """
        Генерирование закрытого ключа
        """
        
        return random.randint(int(2), int(self.order - 2))
    
    
    def get_public_key(self, private_key):
        """
        Получение открытого ключа по закрытому
        """
        
        return self.exponentiate(private_key, self._generator)
    
    
    def exponentiate(self, scalar, element):
        """
        Возведение в степень элемента группы
        """
        
        return self._group(int(gmpy2.powmod(int(element), int(scalar), self.order)))
    
    
    def element_to_bytes(self, element):
        """
        Конвертация элемента группы в байтовое представление
        """
        
        return number.long_to_bytes(int(element))
    
    
    @property
    def group(self):
        """
        Получение группы
        """
        
        return self._group
    
        
    @property
    def generator(self):
        """
        Получение образующего элемента
        """
        
        return self._generator
    
    
    @property
    def order(self):
        """
        Получение порядка группы
        """
        
        return self._modulus

In [11]:
class EllipticCurveDiffieProcessor(object):
    """
    Обработчик обмена ключами по протоколу Диффи-Хеллмана
    в группе точек эллиптической кривой
    """
    
    def __init__(self, curve=None, generator=None):
        """
        Инициализация - выбор кривой, образующего элемента
        Возможно конструирование по готовой кривой и образующему
        """
        
        #
        # Для простоты буду использовать кривую из ГОСТ 34.10 
        #
        
        self._curve = gost3410.CURVES['id-tc26-gost-3410-2012-512-paramSetA'] if curve is None else curve
        self._generator = (self._curve.x, self._curve.y) if generator is None else generator
        self._modulus = self._curve.p
        
        trace('[EllipticCurveDiffieProcessor]', 
              f'Using group of order {self._modulus} over curve {self._curve.name}')
        
    
    def generate_private_key(self):
        """
        Генерирование закрытого ключа
        """
        
        return random.randint(int(2), int(self.order - 2))
    
    
    def get_public_key(self, private_key):
        """
        Получение открытого ключа по закрытому
        """
        
        return self.exponentiate(private_key, self._generator)
    
    
    def exponentiate(self, scalar, element):
        """
        Умножение точки на скаляр
        """
        
        return multiply_by_scalar(self._curve, scalar, element)
    
    
    def element_to_bytes(self, element):
        """
        Конвертация элемента группы в байтовое представление
        """
        
        return gost3410.pub_marshal(element)
    
    
    @property
    def group(self):
        """
        Получение группы
        """
        
        return self._curve
    
    
    @property
    def generator(self):
        """
        Получение образующего элемента
        """
        
        return self._generator
    
    
    @property
    def order(self):
        """
        Получение порядка группы
        """
        
        return self._modulus

----

In [12]:
cipher_suites = ['ECDHE_KUZNYECHIK_MGM_STREEBOG', 'DHE_AES_GCM_SHA256']

In [13]:
def resolve_cipher_suite(cipher_suite):
    """
    Получение примитивов по строковому идентификатору
    """
        
    #
    # Пока захардкодил это все
    #
        
    if cipher_suite == 'ECDHE_KUZNYECHIK_MGM_STREEBOG':
        return EllipticCurveDiffieProcessor, KuznyechikMGM, Streebog
        
    elif cipher_suite == 'DHE_AES_GCM_SHA256':
        return DiffieHellmanProcessor, AESGCM, SHA256
        
    raise ValueError(f'Unsupported cipher suite {cipher_suite}')

----

In [14]:
def hmac(key, *args, hasher=Streebog):
    """
    Вычисление HMAC (согласно Р 50.1.113 - 2016)
    Информационная технология
    КРИПТОГРАФИЧЕСКАЯ ЗАЩИТА ИНФОРМАЦИИ
    Криптографические алгоритмы, сопутствующие применению алгоритмов 
    электронной цифровой подписи и функции хэширования
    """
    
    def xor(lhs, rhs):
        return bytes([l ^^ r for l, r in zip(lhs, rhs)])
    
    IPAD = b'\x36' * 64
    OPAD = b'\x5c' * 64
    
    K_star = key + (b'\x00' * (512 - len(key)))
    
    inner_hash = hash_digest(hasher, xor(K_star, IPAD), *args)
    return hash_digest(hasher, xor(K_star, OPAD), inner_hash)

In [15]:
def test_hmac():
    """
    Р 50.1.113 - 2016
    Информационная технология
    КРИПТОГРАФИЧЕСКАЯ ЗАЩИТА ИНФОРМАЦИИ
    Криптографические алгоритмы, сопутствующие применению алгоритмов 
    электронной цифровой подписи и функции хэширования
    
    4.1.1 HMAC_GOSTR3411_2012_256
    
    Приложение А. Контрольные примеры
    """
    
    K = b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f' + \
        b'\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f'
    
    T = b'\x01\x26\xbd\xb8\x78\x00\xaf\x21\x43\x41\x45\x65\x63\x78\x01\x00'
    
    expected = b'\xa1\xaa\x5f\x7d\xe4\x02\xd7\xb3\xd3\x23\xf2\x99\x1c\x8d\x45\x34' + \
        b'\x01\x31\x37\x01\x0a\x83\x75\x4f\xd0\xaf\x6d\x7c\xd4\x92\x2e\xd9'
    
    assert hmac(K, T, hasher=Streebog) == expected

In [16]:
#
# Протестирую HMAC на контрольном примере из рекомендаций по стандартизации
#

test_hmac()

----

In [17]:
class GOST3410Signer(object):
    """
    Объект-создатель подписей по ГОСТ 34.10 - 2012
    """
    
    def __init__(self):
        """
        Инициализация кривой и ключевой пары
        """
        
        self._curve = gost3410.CURVES['id-tc26-gost-3410-2012-512-paramSetA']
        self._private_key = gost3410.prv_unmarshal(get_random_bytes(64))
        self._public_key = gost3410.public_key(self._curve, self._private_key)
        
        
    def sign(self, *args):
        """
        Подпись произвольного набора байтовых объектов
        """
        
        digest = hash_digest(Streebog, *args)
        return gost3410.sign(self._curve, self._private_key, digest)
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа
        """
        
        return self._public_key
    
    
    @property
    def private_key(self):
        """
        Получение закрытого ключа
        """
        
        return self._private_key

    
    @property
    def curve(self):
        """
        Получение используемой кривой
        """
        
        return self._curve
    
    
    @property
    def generator(self):
        """
        Получение образующего группы точек кривой
        """
        
        return self._curve.x, self._curve.y

In [18]:
def verify_signature(curve, public_key, signature, *args):
    """
    Проверка подписи по ГОСТ 34.10 - 2012
    """
    
    digest = hash_digest(Streebog, *args)
    return gost3410.verify(curve, public_key, digest, signature)

----

## Реализация участников протокола

In [19]:
Offer = namedtuple('Offer', ['cipher_suite', 'group', 'generator'])

In [20]:
class GenericParticipant(object):
    """
    Класс-абстракция над участником протокола, который получает сертификат на свой открытый
    ключ и может осуществлять подпись сообщений
    """
    
    def __init__(self, ca, *, fake_data=None, demo_cert_error=False):
        """
        Инициализация - получение идентификатора, генерирование ключевой пары, 
        получение сертификата
        """
        
        self._id = uuid.uuid4()
        self._signer = GOST3410Signer()
        
        if fake_data is not None:
            #
            # Демонстрация неудачного выполнения протокола
            #
            
            if demo_cert_error:
                #
                # Тут демонстрация неудачи получения серификата
                #
                
                self._signer._public_key = fake_data
                self._certificate = ca.issue_certificate(self)
            else:
                #
                # Тут как бы сертификат и открытый ключ правильные, а закрытый ключ противник не угадал
                #
                
                self._certificate = ca.issue_certificate(self)
                self._signer._public_key = fake_data
        else:
            #
            # Стандартное выполнение
            #
            
            self._certificate = ca.issue_certificate(self)
        
        self._n = None
        
        trace('[GenericParticipant]', f'Certificate = {self._certificate}')
        
        
    def sign(self, *args):
        """
        Подписывание произвольных данных
        """
        
        return self._signer.sign(*args)
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа подписи
        """
        
        return self._signer.public_key
    
    
    @property
    def identifier(self):
        """
        Получение идентификатора
        """
        
        return self._id
    
    
    @property
    def certificate(self):
        """
        Получение сертификата
        """
        
        return self._certificate
    
    
    @property
    def curve(self):
        """
        Получение кривой, используемой для подписи
        """
        
        return self._signer.curve
    
    
    def _get_point_for_schnorr_proof(self):
        #
        # Костыль, но все же работает
        #
        
        self._n = gost3410.prv_unmarshal(get_random_bytes(64))
        return multiply_by_scalar(self._signer.curve, 
                                  self._n, self._signer.generator)
    
    
    def _get_response(self, chellenge):
        return self._n + chellenge * self._signer.private_key

In [21]:
class TLS13Participant(GenericParticipant):
    """
    Обобщенный участник общения по TLS 1.3
    """
    
    def __init__(self, ca, *, fake_data=None, demo_cert_error=False):
        """
        Инициализация - по факту просто сохраняю ссылку на СА
        """
        
        super(TLS13Participant, self).__init__(ca, 
                                               fake_data=fake_data, 
                                               demo_cert_error=demo_cert_error)
        self._ca = ca
    
    
    def check_certificate(self, identifier: uuid.UUID, public_key, cert: Certificate):
        """
        Проверка полученного сертификата
        """
        
        if self._ca.is_revoked(cert):
            trace('[TLS13Participant]', f'Revoked certificate')
            return False
            
        id_as_bytes = str(identifier).encode()
        pk_as_bytes = gost3410.pub_marshal(public_key)
        
        if not verify_signature(self._ca.curve, 
                                self._ca.public_key, 
                                cert.signature, 
                                id_as_bytes, 
                                pk_as_bytes):
            trace('[TLS13Participant]', f'Invalid certificate')
            return False
        
        return True
        
        
    @staticmethod
    def check_mac(key, mac, *args):
        """
        Проверка имитовставки
        """
        
        return mac == hmac(key, *args)
    
    
    @staticmethod
    def check_signature(curve, public_key, signature, *args):
        """
        Проверка подписи
        """
        
        return verify_signature(curve, public_key, signature, *args)

In [22]:
class TLS13Client(TLS13Participant):
    """
    Клиент для TLS 1.3
    """
        
    S2C = 0
    C2S = 1
    
    class AuthMode(Enum):
        """
        Режим аутентицикации
        """
        
        OWA = 1  # Односторонняя
        TWA = 2  # Двусторонняя
    
    
    def __init__(self, ca, cipher_suite, *, fake_data=None, demo_cert_error=False):
        """
        Инициализация - разбор набора примитивов
        """
        
        super(TLS13Client, self).__init__(ca, fake_data=fake_data, 
                                          demo_cert_error=demo_cert_error)
        
        self._key_exchange_type, self._cipher_type, self._hasher = resolve_cipher_suite(cipher_suite)
        self._cipher_suite = cipher_suite
        self._session_keys = {}
        
        trace('[TLS13Client]', f'Using cipher suite {cipher_suite}')
        
        
    def establish_connection(self, server, auth_mode: AuthMode):
        """
        Установление соединения с сервером
        """
        
        key_exchange = self._key_exchange_type()
        
        ephemeral_dh_sk = key_exchange.generate_private_key()
        ephemeral_dh_pk = key_exchange.get_public_key(ephemeral_dh_sk)
        client_nonce    = get_random_bytes(64)
        
        trace('[TLS13Client]', f'Ephemeral DH public key: {ephemeral_dh_pk}')
        trace('[TLS13Client]', f'Nonce: {hexlify(client_nonce)}')
        
        offer = Offer(self._cipher_suite, 
                      key_exchange.group, 
                      key_exchange.generator)
        
        #
        # А теперь запрос на сервер
        #
        
        is_one_way = auth_mode == TLS13Client.AuthMode.OWA
        server_hello = server.get_hello(self.identifier, ephemeral_dh_pk, client_nonce, 
                                        offer, is_one_way)
        
        if server_hello is None:
            trace('[TLS13Client]', f'Server refused the connection')
            return False
        
        #
        # Распакую ответ
        #
        
        ephemeral_dh_server_pk, server_nonce, cipher_suite, c1, c2, c3, c4 = server_hello
        
        if cipher_suite != self._cipher_suite:
            trace('[TLS13Client]', f'Unsupported cipher suite {cipher_suite}')
            return False
        
        #
        # Общий секрет DH
        #
        
        ephemeral_dh_common = key_exchange.exponentiate(ephemeral_dh_sk, ephemeral_dh_server_pk)
        
        #
        # Вырабатываю ключи для симметричного шифрования и имитовставки
        #
        
        client_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_pk),
                                  client_nonce, str(offer).encode()])
        
        server_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_server_pk),
                                  server_nonce, cipher_suite.encode()])
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet])
        
        encryption_key, mac_key = HKDF(key_derivation_data, self._cipher_type.KEY_SIZE, 
                                       b'\x00' * 16, self._hasher, 2)
        
        #
        # Расшифровываю полученные пакеты
        #
        
        cipher = self._cipher_type(encryption_key)
        
        try:
            cert_requested     = cipher.decrypt(*c1)
            server_certificate = cipher.decrypt(*c2)
            server_signature   = cipher.decrypt(*c3)
            server_mac         = cipher.decrypt(*c4)
        except ValueError:
            trace('[TLS13Client]', f'Malformed server response')
            return False
        
        #
        # Проверяю полученнные данные
        #
        
        if is_one_way == cert_requested:
            #
            # Запросили сертификат в ходе односторонней аутентификации или наоборот
            #
            
            trace('[TLS13Client]', f'Invalid client certificate request')
            return False
        
        if not self.check_certificate(server.identifier, server.public_key, 
                                      pickle.loads(server_certificate)):
            trace('[TLS13Client]', f'Invalid server certificate')
            return False
        
        if not TLS13Participant.check_signature(server.curve, server.public_key, server_signature, 
                                                client_packet, server_packet, c1[1], c2[1]):
            trace('[TLS13Client]', f'Malformed server response')
            return False
        
        if not TLS13Participant.check_mac(mac_key, server_mac, 
                                          client_packet, server_packet, c1[1], c2[1], c3[1]):
            trace('[TLS13Client]', f'Malformed server response')
            return False
            
        #
        # Если требуется двусторонняя аутентификация, то зашлю на сервер сертификат
        #
        
        if not is_one_way:
            c5 = cipher.encrypt(pickle.dumps(self.certificate))
            
            signature = self.sign(client_packet, server_packet, c1[1], c2[1], c3[1], c4[1], c5[1])
            c6 = cipher.encrypt(signature)
            
            mac = hmac(mac_key, client_packet, server_packet, c1[1], c2[1], c3[1], c4[1], c5[1], c6[1])
            c7 = cipher.encrypt(mac)
            
            if not server.process_client_certificate(self, c5, c6, c7):
                trace('[TLS13Client]', f'Server refused the connection')
                return False
        
        #
        # Теперь я генерирую ключи шифрования
        #
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet, c1[1], c2[1], c3[1], c4[1]])
            
        self._session_keys[server.identifier] = HKDF(key_derivation_data, self._cipher_type.KEY_SIZE,
                                                     b'\x00' * 16, self._hasher, 2)
        
        trace('[TLS13Client]', f'Server accepted the connection')
        
        trace('[TLS13Client]', 
              f's -> c key: {hexlify(self._session_keys[server.identifier][TLS13Client.S2C])}')
        trace('[TLS13Client]', 
              f'c -> s key: {hexlify(self._session_keys[server.identifier][TLS13Client.C2S])}')
        
        return True
    
    
    def send_wait_receive_message(self, server, message: str):
        """
        Отправка сообщения на сервер и получение ответа
        """
        
        if server.identifier not in self._session_keys:
            trace('[TLS13Client]', f'Connection with server {server.identifier} is not established yet')
            return
        
        cipher = self._cipher_type(self._session_keys[server.identifier][TLS13Client.C2S])
        
        response = server._receive_message_and_respond(self, cipher.encrypt(message.encode()))
        if response is None:
            trace('[TLS13Client]', f'Malformed server response')
        
        try:
            cipher = self._cipher_type(self._session_keys[server.identifier][TLS13Client.S2C])
            return cipher.decrypt(*response).decode()
        except ValueError:
            trace('[TLS13Client]', f'Malformed server response')
            
            
    def change_keys(self, server):
        """
        Смена ключей по инициативе клиента
        """
        
        if server.identifier not in self._session_keys:
            trace('[TLS13Client]', f'Connection with server {server.identifier} is not established yet')
            return
        
        #
        # Меняю у себя
        #
        
        self._change_keys_internal(server.identifier)
        
        #
        # Меняю на сервере
        #
        
        server._change_keys_internal(self.identifier)
        
        
    def _change_keys_internal(self, server_id):
        s2c_key = HKDF(self._session_keys[server_id][TLS13Client.S2C], self._cipher_type.KEY_SIZE, 
                       b'\x00' * 16, self._hasher, 1)
        
        c2s_key = HKDF(self._session_keys[server_id][TLS13Client.C2S], self._cipher_type.KEY_SIZE, 
                       b'\x00' * 16, self._hasher, 1)
        
        trace('[TLS13Client]', f'New s -> c key: {hexlify(s2c_key)}')
        trace('[TLS13Client]', f'New c -> s key: {hexlify(c2s_key)}')
        
        self._session_keys[server_id] = (s2c_key, c2s_key)

In [23]:
class TLS13Server(TLS13Participant):
    """
    Сервер для TLS 1.3
    """
    
    CIPHER_TYPE            = 0
    HASHER                 = 1
    HANDSHAKE_ENC_KEY      = 2
    HANDSHAKE_MAC_KEY      = 3
    HANDSHAKE_VALUES       = 4
    HANDSHAKE_KEY_EXCHANGE = 5
    
    S2C = 0
    C2S = 1
    
    
    def __init__(self, ca, cipher_suites):
        """
        Инициализация - сохранение настроек
        """
        
        super(TLS13Server, self).__init__(ca)
        
        self._cipher_suites = cipher_suites
        self._session_keys  = {}
        self._mapped_states = {}
        
        
    def get_hello(self, client_id, ephemeral_dh_client_pk, client_nonce, offer, is_one_way):
        """
        Получение ServerHello
        """
        
        #
        # Провверяю, что поддерживается вообще набор примитивов
        #
        
        cipher_suite, group, generator = offer
        
        if cipher_suite not in self._cipher_suites:
            trace('[TLS13Server]', f'Unsupported cipher suite {cipher_suite}')
            return None
        
        trace('[TLS13Server]', f'Using cipher suite {cipher_suite} with client {client_id}')
        
        #
        # Получаю примитивы
        #
        
        key_exchange_type, cipher_type, hasher = resolve_cipher_suite(cipher_suite)
        key_exchange = key_exchange_type(group, generator)
        
        #
        # Вырабатываю ключи DH и nonce, после чего получаю общий секрет DH
        #
        
        ephemeral_dh_sk = key_exchange.generate_private_key()
        ephemeral_dh_pk = key_exchange.get_public_key(ephemeral_dh_sk)
        server_nonce    = get_random_bytes(64)
        
        trace('[TLS13Server]', f'Ephemeral DH public key: {ephemeral_dh_pk}')
        trace('[TLS13Server]', f'Nonce: {hexlify(client_nonce)}')
        
        ephemeral_dh_common = key_exchange.exponentiate(ephemeral_dh_sk, ephemeral_dh_client_pk)
        
        #
        # Вырабатываю ключи для симметричного шифрования и имитовставки
        #
        
        client_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_client_pk),
                                  client_nonce, str(offer).encode()])
        
        server_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_pk),
                                  server_nonce, cipher_suite.encode()])
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet])
        
        encryption_key, mac_key = HKDF(key_derivation_data, cipher_type.KEY_SIZE, 
                                       b'\x00' * 16, hasher, 2)
        
        #
        # Рассчитываю нужные значения
        # 
        
        cipher = cipher_type(encryption_key)
        
        cert_request = bytes([not is_one_way])
        c1 = cipher.encrypt(cert_request)
        
        cert = pickle.dumps(self.certificate)
        c2 = cipher.encrypt(cert)
        
        signature = self.sign(client_packet, server_packet, c1[1], c2[1])
        c3 = cipher.encrypt(signature)
        
        mac = hmac(mac_key, client_packet, server_packet, c1[1], c2[1], c3[1])
        c4 = cipher.encrypt(mac)
        
        #
        # Тут будут сохранены нужные значения
        #
        
        self._mapped_states[client_id] = [None, None, None, None, None, None]
        
        #
        # Теперь выработаю ключи (если это односторонняя аутентификация)
        #
        
        if is_one_way:
            self._session_keys[client_id] = TLS13Server._generate_keys(key_exchange, cipher_type.KEY_SIZE, 
                                                                       hasher, ephemeral_dh_common, 
                                                                       client_packet, server_packet, 
                                                                       c1, c2, c3, c4)
        
        #
        # Сохраняю остальное и возвращаю клиенту
        #
        
        self._mapped_states[client_id][TLS13Server.CIPHER_TYPE]       = cipher_type
        self._mapped_states[client_id][TLS13Server.HASHER]            = hasher
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_ENC_KEY] = encryption_key
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_MAC_KEY] = mac_key
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_VALUES]  = (ephemeral_dh_common, client_packet, 
                                                                         server_packet, c1, c2, c3, c4)
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_KEY_EXCHANGE] = key_exchange
        
        return ephemeral_dh_pk, server_nonce, cipher_suite, c1, c2, c3, c4
    
    
    def process_client_certificate(self, client, c5, c6, c7):
        """
        Проверка сертификата клиента
        """
        
        client_id = client.identifier
        
        #
        # Достану ранее сохраненные параметры
        #
        
        cipher_type    = self._mapped_states[client_id][TLS13Server.CIPHER_TYPE]
        hasher         = self._mapped_states[client_id][TLS13Server.HASHER]
        encryption_key = self._mapped_states[client_id][TLS13Server.HANDSHAKE_ENC_KEY]
        mac_key        = self._mapped_states[client_id][TLS13Server.HANDSHAKE_MAC_KEY]
        
        ephemeral_dh_common, client_packet, server_packet, c1, c2, c3, c4 = \
            self._mapped_states[client_id][TLS13Server.HANDSHAKE_VALUES]
        
        key_exchange   = self._mapped_states[client_id][TLS13Server.HANDSHAKE_KEY_EXCHANGE]
        
        #
        # Расшифрую полученный пакет
        #
        
        cipher = cipher_type(encryption_key)
        
        try:
            client_certificate = cipher.decrypt(*c5)
            client_signature   = cipher.decrypt(*c6)
            client_mac         = cipher.decrypt(*c7)
        except ValueError:
            trace('[TLS13Server]', f'Malformed client response')
            return False
        
        #
        # проверю все полученные данные
        #
        
        if not self.check_certificate(client.identifier, client.public_key, 
                                      pickle.loads(client_certificate)):
            trace('[TLS13Server]', f'Invalid client certificate')
            return False
        
        if not TLS13Participant.check_signature(client.curve, client.public_key, client_signature, 
                                                client_packet, server_packet, 
                                                c1[1], c2[1], c3[1], c4[1], c5[1]):
            trace('[TLS13Server]', f'Malformed client response')
            return False
        
        if not TLS13Participant.check_mac(mac_key, client_mac, client_packet, server_packet, 
                                          c1[1], c2[1], c3[1], c4[1], c5[1], c6[1]):
            trace('[TLS13Server]', f'Malformed client response')
            return False
        
        #
        # Все чудесно - генерирую ключи
        #
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet, c1[1], c2[1], c3[1], c4[1]])
            
        self._session_keys[client_id] = TLS13Server._generate_keys(key_exchange, cipher_type.KEY_SIZE, 
                                                                   hasher, ephemeral_dh_common, client_packet, 
                                                                   server_packet, c1, c2, c3, c4)
        
        return True
            
            
    def change_keys(self, client):
        """
        Смена ключей по инициативе сервера
        """
        
        if client.identifier not in self._session_keys:
            trace('[TLS13Client]', f'Connection with client {client.identifier} is not established yet')
            return
        
        #
        # Меняю у себя
        #
        
        self._change_keys_internal(client.identifier)
        
        #
        # Меняю на сервере
        #
        
        client._change_keys_internal(self.identifier)
        
        
    def _change_keys_internal(self, client_id):
        s2c_key = HKDF(self._session_keys[client_id][TLS13Client.S2C], 
                       self._mapped_states[client_id][TLS13Server.CIPHER_TYPE].KEY_SIZE, 
                       b'\x00' * 16, self._mapped_states[client_id][TLS13Server.HASHER], 1)
        
        c2s_key = HKDF(self._session_keys[client_id][TLS13Client.C2S], 
                       self._mapped_states[client_id][TLS13Server.CIPHER_TYPE].KEY_SIZE, 
                       b'\x00' * 16, self._mapped_states[client_id][TLS13Server.HASHER], 1)
        
        trace('[TLS13Client]', f'New s -> c key: {hexlify(s2c_key)}')
        trace('[TLS13Client]', f'New c -> s key: {hexlify(c2s_key)}')
        
        self._session_keys[client_id] = (s2c_key, c2s_key)
    
    
    def _receive_message_and_respond(self, client, encrypted_message):
        client_id = client.identifier
        
        try:
            cipher = self._mapped_states[client_id][TLS13Server.CIPHER_TYPE](
                self._session_keys[client_id][TLS13Server.C2S])
            
            message = cipher.decrypt(*encrypted_message).decode()
            print(f'Received message "{message}"')
            
            cipher = self._mapped_states[client_id][TLS13Server.CIPHER_TYPE](
                self._session_keys[client_id][TLS13Server.S2C])
            
            return cipher.encrypt(f'Hello, {message}!'.encode())
        except ValueError:
            trace('[TLS13Server]', f'Malformed client message')
    
    
    @staticmethod
    def _generate_keys(key_exchange, key_size, hasher, ephemeral_dh_common, 
                       client_packet, server_packet, c1, c2, c3, c4):
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet, c1[1], c2[1], c3[1], c4[1]])
            
        s2c_key, c2s_key = HKDF(key_derivation_data, key_size, 
                                b'\x00' * 16, hasher, 2)
        
        trace('[TLS13Server]', f's -> c key: {hexlify(s2c_key)}')
        trace('[TLS13Server]', f'c -> s key: {hexlify(c2s_key)}')
        
        return s2c_key, c2s_key

In [24]:
class TLS13CertificationAuthority(object):
    """
    Удостоверющий центр для TLS 1.3
    """
    
    def __init__(self):
        """
        Инициализация - генерирование ключевой пары
        """
        
        self._signer = GOST3410Signer()
        
        self._certificates = set()
        self._revoked_certificates = set()
        
        trace('[TLS13CertificationAuthority]', 
              f'Public key = {self._signer.public_key}')
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа
        """
        
        return self._signer.public_key
    
    
    @property
    def curve(self):
        """
        Получение кривой, используемой для подписи
        """
        
        return self._signer.curve
        
        
    def issue_certificate(self, participant: GenericParticipant):
        """
        Выдача сертификата на открытый ключ
        """
        
        #
        # Проверю, что клиент действительно владеет закрытым ключом
        #
        
        if not self._check_participant(participant):
            trace('[TLS13CertificationAuthority]', 
                  f'Participant cannot prove a valid private key possession')
            
            return None
        
        #
        # Все чудесно, подписываю ключ и отдаю сертификат
        #
        
        id_as_bytes = str(participant.identifier).encode()
        pk_as_bytes = gost3410.pub_marshal(participant.public_key)
        
        cert = Certificate(participant.identifier, participant.public_key, 
                           self._signer.sign(id_as_bytes, pk_as_bytes))
        
        self._certificates.add(cert)
        
        trace('[TLS13CertificationAuthority]', 
              f'Issued certificate for {participant.identifier}. Total certificates: {len(self._certificates)}')
        
        return cert
    
    
    def revoke_certificate(self, participant: GenericParticipant):
        """
        Отзыв сертификата
        """
        
        #
        # А можно ли вообще отзывать сертификат?
        #
        
        if not self._check_participant(participant):
            trace('[TLS13CertificationAuthority]', 
                  f'Participant cannot prove a valid private key possession')
            
            return 
        
        if participant.certificate not in self._certificates:
            #
            # А отзывать то и нечего
            #
            
            return
            
        self._revoked_certificates.add(participant.certificate)
        self._certificates.discard(participant.certificate)
        
        trace('[TLS13CertificationAuthority]', 
              f'List of revoked certificates now has {len(self._revoked_certificates)} elements')
        
        
    def is_revoked(self, cert: Certificate):
        """
        Проверка на факт отзыва сертификата
        """
        
        return cert in self._revoked_certificates
    
    
    def _check_participant(self, participant: GenericParticipant) -> bool:
        Q = participant._get_point_for_schnorr_proof()
        c = gost3410.prv_unmarshal(get_random_bytes(64))
        t = participant._get_response(c)
        
        tP  = multiply_by_scalar(self._signer.curve, t, self._signer.generator)
        cPK = multiply_by_scalar(self._signer.curve, c, participant.public_key)
        
        return tP == add_points(self._signer.curve, Q, cPK)

----
## Демонстрация удачного выполнения протоколов

In [25]:
# Создание удостоверяющего центра
ca = TLS13CertificationAuthority()

[TRACE] [TLS13CertificationAuthority] Public key = (3547261982440344541504391960435830580562851827350298619348226904427246852393452448044062221653366795945934009150207017061367691411965791647310607664614408, 1844250996387209465458253201980094411280327310881412568664370264830990751867146830505133800890373714887369062521557118012048738243684524537024029565987509)


In [26]:
# Создание сервера
srv = TLS13Server(ca, cipher_suites)

[TRACE] [TLS13CertificationAuthority] Issued certificate for b08b571c-867c-4b76-9632-08954979912d. Total certificates: 1
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('b08b571c-867c-4b76-9632-08954979912d'), public_key=(1255081151011809304703655938623031101909732541638904528412711059346630467560354452693586343253276812745810791684813453334007464393687616450557689616028347, 10550844824748112555567450075234062202252750235533336009362665246265728157472209209323756378039505087671081149214759175384276003062232666441446509216116905), signature=b"I\xac\xe6\x99'\x1a\xd0\xbe\xfb{\xddI\xce^\x89N\x95\xf3\x9f\xd2\x1f\x02\xe51%\xdfi\xbaY\xeb\xc3H\x19\xb5\x84\x92\xf5Cu\xff\xc7\x8f\x84@\xbb<o\xbb\xcf\x7f/\xd9/\x99\xf6\xcfp6\xf9\x85d{\x1cJP\xf3\xf7[\x896\x91\xb8\xdb\xcd\xfd\x84+d\x1aTQ\xe4\xf2\x97\x85\xc1\xd1\xe8\xc9t\xf3\xab\xb4\x06\xb1\xcepr\x1f`\xa8\xc6\xfb\xd9\xb9\xe8\x88.\x13\x04\x0cknoPZF\xcb\xcf\x82\xa0\xfc\xc4f\xafQ\x85A")


In [27]:
# Первый валидный пользователь (использует ECDH, Кузнечик в режиме MGM и Стрибог)
alice = TLS13Client(ca, cipher_suite=cipher_suites[0])

[TRACE] [TLS13CertificationAuthority] Issued certificate for 136c98ee-e340-4bdd-8db6-d9d3161eabff. Total certificates: 2
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('136c98ee-e340-4bdd-8db6-d9d3161eabff'), public_key=(5740001772319891585661288704945207633447067468655287151431626446315114289890338068700245482267225500184694781322566930607850834466882205251016494404732659, 10713471306414883409362846172477222984348626136063200942936739397519536076246496879339378249665005740230258751922356784019317205841500773592700902662383242), signature=b'0h\x974\n8\xc5f \xb5i\xaf\xf0\x08A[\x05`\t\xb4q\x9c\xf1\xbf\xb55?=\x00\x08ow\x87\x82A(\xa3\x84@\xba(\xf0\xdc_\xfd;5\x970^f\xfe\xdb_\xee\xdc\x91t-\xcd4*\x1b\xe6o\xd4\x86\xa8R\x18\x02\xf2\xf1\xfe\x87\x93\x86\xcc\t\x87#\xb5:Z\x8b4aw)\xcc\xc2\xbc&\x83\xdb\xf4!\xbf\x9a$8\x03U\xf0\xcc\xfecTiI)\xfd\x02\xad0g\xfd\xa7\x04\xbd\x7fY\xa3\x1a!v\xd1%')
[TRACE] [TLS13Client] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG


In [28]:
# Второй валидный пользователь (использует DH, AES в режиме GCM и SHA256)
bob = TLS13Client(ca, cipher_suite=cipher_suites[1])

[TRACE] [TLS13CertificationAuthority] Issued certificate for 578cbd97-0412-4ebd-99fa-fd5afa043c2a. Total certificates: 3
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('578cbd97-0412-4ebd-99fa-fd5afa043c2a'), public_key=(12243723780192037873993687991557638896556875533396547998064618742741252087584133234833016923102235936969461323780167734620443262724136699526014326405815813, 11538055713357095453970752545713785998366835141693730549322682397110788860672517441525293230058813335854664258998246278461481843229811178632039337922026107), signature=b'L\x1f\x03\x14\x001^\x07\x9e;\xd4uq\xbe\xce\xc6J>\xc2k\x82\\h\xbb\xa3\x8a5w\x87h\xf9\x9f\xb31o$X&\xcc\xf7J\xa9>\xfa*!Ji\xba\xe0,\x81\r\xec\x82\xd3z\t\x19i\xc3\xe3-\xc0X^\xcd\xfa^\xba\xe2D\x85\xecP\xc7k\x8d\xea\xe1\xab\x8d\xeb\x14\x88\xf4\x95U\xa0M*\x99\x07\x1bp\x8e\x14\x84YMBq\xed\x18{\xa3\x10P\xd26\xbb\xbcQjA_\xe7\xabzS8&N\xda\xcf\x07\xedG')
[TRACE] [TLS13Client] Using cipher suite DHE_AES_GCM_SHA256


In [29]:
# Устанавливаю соединение с односторонней аутентификацией
alice.establish_connection(srv, TLS13Client.AuthMode.OWA)

[TRACE] [EllipticCurveDiffieProcessor] Using group of order 13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006083527 over curve id-tc26-gost-3410-2012-512-paramSetA
[TRACE] [TLS13Client] Ephemeral DH public key: (2906144136355548196906378010282450870946904682260249985511198580509478390104785205222244047349459747887064661887762812271875945794052866827728307965841601, 2532843639089105055095989541019553399863141248564346224358420698071089833330459102553991034476621420136074711713124945589947655579479829777998478840434652)
[TRACE] [TLS13Client] Nonce: b'f9704737dd170210eb4868d09b811bfa0c42900ecc64f1169ae4f41f29ee90459af6d92bea47891f0aa1a02bbc4a4c5def334ea26e1ec14c2578757d44c006ed'
[TRACE] [TLS13Server] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG with client 136c98ee-e340-4bdd-8db6-d9d3161eabff
[TRACE] [EllipticCurveDiffieProcessor] Using group of order 13407807929942597099574024998205846

True

In [30]:
# Устанавливаю соединение с двусторонней аутентификацией
bob.establish_connection(srv, TLS13Client.AuthMode.TWA)

[TRACE] [DiffieHellmanProcessor] Using group of order 12370029342359802523755317725262925541059624647213414303808301449998527392949992237225263693247979275003144387520650007527343654288371161689355949459089863
[TRACE] [TLS13Client] Ephemeral DH public key: 8233515588671401333998574085084454549228210808007563701932907100305278823103559609285416714756377517886520251497915103503971064519425634285197751897994836
[TRACE] [TLS13Client] Nonce: b'6c2391a3f3643c9fd11966062b11c8be6ae656c462961122ba0018fa8fa91f5b5a9e789b543dd6f7d4b6b0590523de8114eeccb1710689ee7808cc55c67ae1a2'
[TRACE] [TLS13Server] Using cipher suite DHE_AES_GCM_SHA256 with client 578cbd97-0412-4ebd-99fa-fd5afa043c2a
[TRACE] [DiffieHellmanProcessor] Using group of order 12370029342359802523755317725262925541059624647213414303808301449998527392949992237225263693247979275003144387520650007527343654288371161689355949459089863
[TRACE] [TLS13Server] Ephemeral DH public key: 1945229989877227658585308799599777028660523739250924239774542

True

In [31]:
# Отправлю сообщение
alice.send_wait_receive_message(srv, 'world')

Received message "world"


'Hello, world!'

In [32]:
# Сменю ключи по инициативе клиента
alice.change_keys(srv)

[TRACE] [TLS13Client] New s -> c key: b'5a2cf85c6156b4e6d4f8aa7f68e1762ed5596c18c9a535ed1e5ffa1ac4a4727f'
[TRACE] [TLS13Client] New c -> s key: b'6289ac7dadf3eed5db00e74a5ae3b764ebfed74623dc5004f5f5aae35f2df907'
[TRACE] [TLS13Client] New s -> c key: b'5a2cf85c6156b4e6d4f8aa7f68e1762ed5596c18c9a535ed1e5ffa1ac4a4727f'
[TRACE] [TLS13Client] New c -> s key: b'6289ac7dadf3eed5db00e74a5ae3b764ebfed74623dc5004f5f5aae35f2df907'


In [33]:
# И еще раз отправлю сообщение, чтобы было видно, что ключи успешно поменялись
alice.send_wait_receive_message(srv, 'world')

Received message "world"


'Hello, world!'

In [34]:
# Отправлю сообщение другим клиентом
bob.send_wait_receive_message(srv, 'crypto')

Received message "crypto"


'Hello, crypto!'

In [35]:
# Смена ключей по инициативе сервера
srv.change_keys(bob)

[TRACE] [TLS13Client] New s -> c key: b'e078c5451366b79fc91d288ab4e1e409'
[TRACE] [TLS13Client] New c -> s key: b'6c766e2f2fb01502d5f5c25601185f4f'
[TRACE] [TLS13Client] New s -> c key: b'e078c5451366b79fc91d288ab4e1e409'
[TRACE] [TLS13Client] New c -> s key: b'6c766e2f2fb01502d5f5c25601185f4f'


In [36]:
# Все по-прежнему успешно отправляется
bob.send_wait_receive_message(srv, 'crypto')

Received message "crypto"


'Hello, crypto!'

----
## Отзыв сертификата

In [37]:
# Третий валидный пользователь
carol = TLS13Client(ca, cipher_suite=cipher_suites[0])

[TRACE] [TLS13CertificationAuthority] Issued certificate for 2237cf50-5d60-4965-9e9f-b1216730cf36. Total certificates: 4
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('2237cf50-5d60-4965-9e9f-b1216730cf36'), public_key=(10729234633931424846909890963717037020913743823754355813534701136721376845018034619057066708749500080514512652659809983622900920875648651436612923139946247, 4361395650386207385214563874266575965938361885909412320446769094622515799014568819186491698430654436977330264828967248217176188364682325508081471381658055), signature=b'J\x13\xffm\xe8W\x97\xba\xc9\x05\xcbVk\x8a\x9a)\x8c\x87\x0f\x04I+(\xcc\x10\xe0BF\xb1~4\x8c\x97\x08^\xfc\xca+\x93\xde\t\x82R\xed6\x88d\xdfz\xab\x06\xe1\xca\xad\x89?z\xca\xc5\x02\x80\x08\x97\x9a]\xbc}\x16g\t\x1b\xd7h\x14W\x19\xa9W.\x03\xdd\x138=\x9a\xd6\x19\xf3\x13\xb45mt\x11.\x84)\x92\xe6\xbd\x12\xc2\x93&G\x0e%N\xb5\xe8x\xc8w\xd9\xe3\x7f\xe3\xc0$\x8b\xce\x90j\xfcB/\xb9\xb1')
[TRACE] [TLS13Client] Using cipher suite ECDHE_KUZNYECHIK_MGM

In [38]:
# Отзову сертификат Кэрол
ca.revoke_certificate(carol)

[TRACE] [TLS13CertificationAuthority] List of revoked certificates now has 1 elements


In [39]:
# Попробую подключиться, используя двустороннюю аутентификацию
carol.establish_connection(srv, TLS13Client.AuthMode.TWA)

[TRACE] [EllipticCurveDiffieProcessor] Using group of order 13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006083527 over curve id-tc26-gost-3410-2012-512-paramSetA
[TRACE] [TLS13Client] Ephemeral DH public key: (12352726250135081051007645982917382766217728235171211130077112076007884597163237252779724075261899150703469559649778976333313696642519857044904890653492884, 5143411216499265862135713963222783107891101490672043782605251848653513937843553223391372973394733639105014548168509524412962990978651787408473699423500805)
[TRACE] [TLS13Client] Nonce: b'96a19c581fe2efdcd656050b33e5e4694b8339a4506cfa9a84b742c270b1d00fb27dab6cde7ccb78bb297f24ee8109c7028685d06105983f7afc437b6a04a025'
[TRACE] [TLS13Server] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG with client 2237cf50-5d60-4965-9e9f-b1216730cf36
[TRACE] [EllipticCurveDiffieProcessor] Using group of order 1340780792994259709957402499820584

False

----
## Демонстрация неудачного выполнения протоколов

In [40]:
# Тут Мэллори пытается установить подключение с двусторонней аутентификацией,
# используя сертификат не на свой ключ (я просто подменю тут открытый ключ для простоты)
mallory = TLS13Client(ca, cipher_suite=cipher_suites[0], 
                      fake_data=bob.public_key)

[TRACE] [TLS13CertificationAuthority] Issued certificate for f0b41c7f-4da0-457a-a629-61f2b0b68d52. Total certificates: 4
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('f0b41c7f-4da0-457a-a629-61f2b0b68d52'), public_key=(12536520645721968834699579319042359506500614261395827965357391933430685089233368015389835710552621581946437680991937996537911049758111619554691119442185393, 7610840890401345646601680484370402787849990096298549587775662941478422193756942218254736575296424303870310386682984201860400047895832325996962545884626661), signature=b"l':wn\xe5T\xe3M\x7fz\xce\xcd\xbb\xad=q\x8f\x05b\xa5\xf5\x8e\xaf\xdc\x91C@\x81%\xa18:\xea\x9f2\x86\x80\xdf_g\x1f!\xa3\xa5\x08\x7f\xe3\xed\x7f\x02}[\x86\xfeG\xf9\xb5\xb1\xae$]\x15\xe6\xb4\x97\xf6\xf1\xae\xe4\x07~\xdc=\x08\xa4\x0fd\x90b0y7\xc0\x97Aa\x16\xb0(\ns\x97\xdfFi\x9e\xae\x8a\xa4\xee\xeb\xdd\xdf\x94\x89_\xf3~n]\x10*J%\x16WR\xe3\x97\xcc\xce\xe9[H\xf4\x0e\x8d")
[TRACE] [TLS13Client] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG


In [41]:
mallory.establish_connection(srv, TLS13Client.AuthMode.TWA)

[TRACE] [EllipticCurveDiffieProcessor] Using group of order 13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006083527 over curve id-tc26-gost-3410-2012-512-paramSetA
[TRACE] [TLS13Client] Ephemeral DH public key: (237942321684377398292392338264186936171751803038157157120964078499303054090331008654342879260506311514348476165098919997427870876604257491348171197598690, 11827194400497527664674980392983484281437250120032555264841078029349786632252829105439836404928622399092356199931931630866451310000959496935357317342668241)
[TRACE] [TLS13Client] Nonce: b'c100caec266910cf56f17435bdca9653bc72a4ac41e9167c72d1d66de9039bab68196bed589729210d5a13a4d79f6f714a71f7f0d0815e926eee6a9e8648acfa'
[TRACE] [TLS13Server] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG with client f0b41c7f-4da0-457a-a629-61f2b0b68d52
[TRACE] [EllipticCurveDiffieProcessor] Using group of order 13407807929942597099574024998205846

False

In [42]:
# Мэллори теперь пытается получить сертификат на ключ Боба, не зная его закрытого ключа
mallory = TLS13Client(ca, cipher_suite=cipher_suites[0], 
                      fake_data=bob.public_key, 
                      demo_cert_error=True)

[TRACE] [TLS13CertificationAuthority] Participant cannot prove a valid private key possession
[TRACE] [GenericParticipant] Certificate = None
[TRACE] [TLS13Client] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG
